<p> Fernanda Kaory - RM551104</p>
<p> Lucas Toledo - RM97913</p>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jinja2 import Template
import base64
from io import BytesIO

In [2]:
data = sns.load_dataset('iris')

In [10]:
def gerar_overview(df):
    estatisticas = {
        'Número de variáveis': df.shape[1],
        'Número de observações': df.shape[0],
        'Células faltantes': df.isnull().sum().sum(),
        'Células faltantes (%)': (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100,
        'Linhas duplicadas': df.duplicated().sum(),
        'Linhas duplicadas (%)': (df.duplicated().sum() / df.shape[0]) * 100

    }

    # Contar tipos de variáveis
    tipos_var = {
        'Numéricas': len(df.select_dtypes(include=[np.number]).columns),
        'Categóricas': len(df.select_dtypes(include=['object']).columns)
    }

    return estatisticas, tipos_var

In [3]:
# Função para gerar estatísticas descritivas ajustadas
def gerar_estatisticas(df):
    estatisticas_numericas = df.describe(include=[np.number]).transpose()  # Estatísticas numéricas
    estatisticas_categoricas = df.describe(include=['object']).transpose()  # Estatísticas categóricas

    # Ajusta os resultados removendo colunas irrelevantes para cada tipo
    estatisticas_numericas = estatisticas_numericas[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
    estatisticas_categoricas = estatisticas_categoricas[['count', 'unique', 'top', 'freq']]

    # Combina as duas tabelas
    estatisticas_combinação = pd.concat([estatisticas_numericas, estatisticas_categoricas], axis=0)

    return estatisticas_combinação

In [4]:
# Função para analisar colunas numéricas
def analisar_numerica(df, coluna):
    estatisticas = {
        'Distintas': df[coluna].nunique(),
        'Distintas (%)': f"{(df[coluna].nunique() / len(df)) * 100:.1f}%",
        'Faltantes': df[coluna].isnull().sum(),
        'Faltantes (%)': f"{(df[coluna].isnull().sum() / len(df)) * 100:.1f}%",
        'Média': df[coluna].mean(),
        'Mínimo': df[coluna].min(),
        'Máximo': df[coluna].max(),
        'Zeros': (df[coluna] == 0).sum(),
        'Zeros (%)': f"{(df[coluna] == 0).sum() / len(df) * 100:.1f}%"

    }

    # Gerar gráfico de distribuição
    plt.figure(figsize=(5, 3))
    sns.histplot(df[coluna].dropna(), bins=20, kde=False)
    plt.title(f'Distribuição de {coluna}')
    plt.tight_layout()

    # Salvar gráfico como imagem
    img = BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    img_b64 = base64.b64encode(img.getvalue()).decode()
    plt.close()

    return estatisticas, img_b64

In [5]:
# Função para analisar colunas categóricas
def analisar_categorica(df, coluna):
    estatisticas = {
        'Distintas': df[coluna].nunique(),
        'Distintas (%)': f"{(df[coluna].nunique() / len(df)) * 100:.1f}%",
        'Faltantes': df[coluna].isnull().sum(),
        'Faltantes (%)': f"{(df[coluna].isnull().sum() / len(df)) * 100:.1f}%"

    }

    # Gerar gráfico de barras
    plt.figure(figsize=(5, 3))
    sns.countplot(y=df[coluna], order=df[coluna].value_counts().index)
    plt.title(f'Distribuição de {coluna}')
    plt.tight_layout()

    # Salvar gráfico como imagem
    img = BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    img_b64 = base64.b64encode(img.getvalue()).decode()
    plt.close()

    return estatisticas, img_b64


In [6]:
# Função para gerar gráficos de dispersão entre variáveis numéricas
def gerar_dispersao(df):
    graficos_dispersao = []
    colunas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
    for i, col1 in enumerate(colunas_numericas):
        for col2 in colunas_numericas[i+1:]:
            plt.figure(figsize=(8, 6))
            sns.scatterplot(x=df[col1], y=df[col2], hue=df['species'], palette='deep')
            plt.title(f'Gráfico de dispersão entre {col1} e {col2}')
            img = BytesIO()
            plt.savefig(img, format='png')
            img.seek(0)
            img_b64 = base64.b64encode(img.getvalue()).decode()
            plt.close()
            graficos_dispersao.append((f'Gráfico de dispersão entre {col1} e {col2}', img_b64))
    return graficos_dispersao


In [7]:
# Função para gerar matriz de correlação (considerando apenas variáveis numéricas)
def gerar_correlacao(df):
    df_numerico = df.select_dtypes(include=['float64', 'int64'])  # Seleciona apenas colunas numéricas
    if df_numerico.shape[1] > 1:  # Apenas gera o gráfico se houver mais de uma variável numérica
        plt.figure(figsize=(10, 8))
        matriz_correlacao = df_numerico.corr()
        sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', linewidths=0.5)
        img = BytesIO()
        plt.savefig(img, format='png')
        img.seek(0)
        img_b64 = base64.b64encode(img.getvalue()).decode()
        plt.close()
        return img_b64
    else:
        return None

In [15]:
def gerar_html(df):
    colunas_numericas = df.select_dtypes(include=[np.number]).columns
    colunas_categoricas = df.select_dtypes(include=['object', 'category']).columns

    # Analisar colunas numéricas
    analise_numerica = []
    for col in colunas_numericas:
        estatisticas, img = analisar_numerica(df, col)
        analise_numerica.append((col, estatisticas, img))

    # Analisar colunas categóricas
    analise_categorica = []
    for col in colunas_categoricas:
        estatisticas, img = analisar_categorica(df, col)
        analise_categorica.append((col, estatisticas, img))

    # Gerar gráficos de dispersão
    graficos_dispersao = gerar_dispersao(df)

    # Gerar matriz de correlação
    matriz_correlacao = gerar_correlacao(df)

    # Gerar visão geral
    estatisticas_gerais, tipos_var = gerar_overview(df)

    # Modelo de HTML para o relatório
    modelo_html = """
    <html>
    <head>
        <title>Relatório de Análise de Dados - Conjunto Iris</title>
        <style>
            body { font-family: Arial, sans-serif; padding: 20px; }
            h1 { color: #16ab20; }
            h2 { color: #074f0c; border-bottom: 2px solid #ddd; padding-bottom: 5px; }
            table { width: 100%; border-collapse: collapse; margin-bottom: 20px; }
            th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
            th { background-color: #16ab20; }
            img { margin: 10px 0; border: 1px solid #ddd; width: 300px; }
            .tabela-estatisticas { margin-bottom: 30px; width: 45%; float: left; margin-right: 5%; }
            .analise-coluna::after { content: ""; clear: both; display: table; }
        </style>
    </head>
    <body>
        <h1>Relatório de Análise de Dados - Conjunto Iris</h1>

        <h2>Visão Geral do Dataset</h2>
        <table>
            <tr><th>Atributo</th><th>Valor</th></tr>
            {% for chave, valor in estatisticas_gerais.items() %}
                <tr><td>{{ chave }}</td><td>{{ valor }}</td></tr>
            {% endfor %}
        </table>

        <h2>Tipos de Variáveis</h2>
        <table>
            <tr><th>Tipo</th><th>Quantidade</th></tr>
            {% for tipo, quantidade in tipos_var.items() %}
                <tr><td>{{ tipo }}</td><td>{{ quantidade }}</td></tr>
            {% endfor %}
        </table>

        <div class="analise-coluna">
            {% for coluna, estatisticas, img in analise_numerica %}
                <div class="tabela-estatisticas">
                    <h2>Análise da Coluna Numérica: {{ coluna }}</h2>
                    <table>
                        {% for chave, valor in estatisticas.items() %}
                            <tr><td>{{ chave }}</td><td>{{ valor }}</td></tr>
                        {% endfor %}
                    </table>
                    <img src="data:image/png;base64,{{ img }}" alt="Gráfico de {{ coluna }}">
                </div>
            {% endfor %}
        </div>
        <div class="analise-coluna">
            {% for coluna, estatisticas, img in analise_categorica %}
                <div class="tabela-estatisticas">
                    <h2>Análise da Coluna Categórica: {{ coluna }}</h2>
                    <table>
                        {% for chave, valor in estatisticas.items() %}
                            <tr><td>{{ chave }}</td><td>{{ valor }}</td></tr>
                        {% endfor %}
                    </table>
                    <img src="data:image/png;base64,{{ img }}" alt="Gráfico de {{ coluna }}">
                </div>
            {% endfor %}
        </div>

        <h2>Gráficos de Dispersão</h2>
        <div class="analise-coluna">
            {% for titulo, img in graficos_dispersao %}
                <div class="tabela-estatisticas">
                    <h3>{{ titulo }}</h3>
                    <img src="data:image/png;base64,{{ img }}" alt="{{ titulo }}">
                </div>
            {% endfor %}
        </div>

        {% if matriz_correlacao %}
        <h2>Matriz de Correlação</h2>
        <img src="data:image/png;base64,{{ matriz_correlacao }}" alt="Matriz de Correlação">
        {% endif %}
    </body>
    </html>
    """

    template = Template(modelo_html)
    html_renderizado = template.render(
        estatisticas_gerais=estatisticas_gerais,
        tipos_var=tipos_var,
        analise_numerica=analise_numerica,
        analise_categorica=analise_categorica,
        graficos_dispersao=graficos_dispersao,
        matriz_correlacao=matriz_correlacao
    )

    # Salvar o relatório em um arquivo HTML
    with open('relatorio_iris.html', 'w', encoding='utf-8') as arquivo_html:
        arquivo_html.write(html_renderizado)

    print("Relatório gerado com sucesso!")

In [16]:
# Gera o relatório HTML
gerar_html(data)

Relatório gerado com sucesso!
